In [2]:
from typing import Dict, Text

import os
import numpy as np
import pandas as pd
import tensorflow as tf
import pprint

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

2024-02-05 15:32:30.195355: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/alexlee/miniconda3/envs/hkjc-tf/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "timestamp": x["timestamp"],
})
movies = movies.map(lambda x: x["movie_title"])

In [12]:
df = pd.read_csv(os.path.join('data','all_races_cleaned.csv'), index_col=0)

ratings = pd.DataFrame()
ratings['movie_title'] = df['horse_colour'].astype(str)
ratings['user_id'] = df['race_class'].astype(str)
ratings['timestamp'] = df['horse_weight'].astype('int64')
ratings['user_rating'] = df['place'].astype('float32')
ratings = tf.data.Dataset.from_tensor_slices(dict(ratings))

ratings

<_TensorSliceDataset element_spec={'movie_title': TensorSpec(shape=(), dtype=tf.string, name=None), 'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'timestamp': TensorSpec(shape=(), dtype=tf.int64, name=None), 'user_rating': TensorSpec(shape=(), dtype=tf.float32, name=None)}>

In [13]:
timestamps = np.concatenate(list(ratings.map(lambda x: x["timestamp"]).batch(100)))

max_timestamp = timestamps.max()
min_timestamp = timestamps.min()

timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,
)

unique_movie_titles = np.unique(np.concatenate(list(movies.batch(1000))))
unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["user_id"]))))

In [14]:
class UserModel(tf.keras.Model):

  def __init__(self, use_timestamps):
    super().__init__()

    self._use_timestamps = use_timestamps

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

    if use_timestamps:
      self.timestamp_embedding = tf.keras.Sequential([
          tf.keras.layers.Discretization(timestamp_buckets.tolist()),
          tf.keras.layers.Embedding(len(timestamp_buckets) + 1, 32),
      ])
      self.normalized_timestamp = tf.keras.layers.Normalization(
          axis=None
      )

      self.normalized_timestamp.adapt(timestamps)

  def call(self, inputs):
    if not self._use_timestamps:
      return self.user_embedding(inputs["user_id"])

    return tf.concat([
        self.user_embedding(inputs["user_id"]),
        self.timestamp_embedding(inputs["timestamp"]),
        tf.reshape(self.normalized_timestamp(inputs["timestamp"]), (-1, 1)),
    ], axis=1)

In [15]:
class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ])

    self.title_vectorizer = tf.keras.layers.TextVectorization(
        max_tokens=max_tokens)

    self.title_text_embedding = tf.keras.Sequential([
      self.title_vectorizer,
      tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
      tf.keras.layers.GlobalAveragePooling1D(),
    ])

    self.title_vectorizer.adapt(movies)

  def call(self, titles):
    return tf.concat([
        self.title_embedding(titles),
        self.title_text_embedding(titles),
    ], axis=1)

In [16]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, use_timestamps):
    super().__init__()
    self.query_model = tf.keras.Sequential([
      UserModel(use_timestamps),
      tf.keras.layers.Dense(32)
    ])
    self.candidate_model = tf.keras.Sequential([
      MovieModel(),
      tf.keras.layers.Dense(32)
    ])
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"],
        "timestamp": features["timestamp"],
    })
    movie_embeddings = self.candidate_model(features["movie_title"])

    return self.task(query_embeddings, movie_embeddings)

In [22]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100, seed=42, reshuffle_each_iteration=False)

split = 0.8

train = shuffled.take(int(split * len(shuffled)))
test = shuffled.skip(int(split * len(shuffled))).take(int((1-split) * len(shuffled)))

cached_train = train.shuffle(100).batch(2048)
cached_test = test.batch(4096).cache()

In [23]:
model = MovielensModel(use_timestamps=False)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Epoch 1/3
9/9 [==============================] - 3s 150ms/step - factorized_top_k/top_1_categorical_accuracy: 0.2354 - factorized_top_k/top_5_categorical_accuracy: 0.2371 - factorized_top_k/top_10_categorical_accuracy: 0.2774 - factorized_top_k/top_50_categorical_accuracy: 0.2796 - factorized_top_k/top_100_categorical_accuracy: 0.2805 - loss: 14379.5846 - regularization_loss: 0.0000e+00 - total_loss: 14379.5846
Epoch 2/3
9/9 [==============================] - 1s 151ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0604 - factorized_top_k/top_5_categorical_accuracy: 0.0650 - factorized_top_k/top_10_categorical_accuracy: 0.0650 - factorized_top_k/top_50_categorical_accuracy: 0.0698 - factorized_top_k/top_100_categorical_accuracy: 0.0742 - loss: 14208.7047 - regularization_loss: 0.0000e+00 - total_loss: 14208.7047
Epoch 3/3
2/2 [==============================] - 0s 111ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0111 - factorized_top_k/top_5_categorical_accuracy: 0.0111

In [24]:
model = MovielensModel(use_timestamps=True)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

train_accuracy = model.evaluate(
    cached_train, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]
test_accuracy = model.evaluate(
    cached_test, return_dict=True)["factorized_top_k/top_100_categorical_accuracy"]

print(f"Top-100 accuracy (train): {train_accuracy:.2f}.")
print(f"Top-100 accuracy (test): {test_accuracy:.2f}.")

Epoch 1/3
9/9 [==============================] - 3s 166ms/step - factorized_top_k/top_1_categorical_accuracy: 0.2983 - factorized_top_k/top_5_categorical_accuracy: 0.3623 - factorized_top_k/top_10_categorical_accuracy: 0.3693 - factorized_top_k/top_50_categorical_accuracy: 0.3857 - factorized_top_k/top_100_categorical_accuracy: 0.3952 - loss: 14462.2150 - regularization_loss: 0.0000e+00 - total_loss: 14462.2150
Epoch 2/3
9/9 [==============================] - 2s 168ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1711 - factorized_top_k/top_5_categorical_accuracy: 0.3963 - factorized_top_k/top_10_categorical_accuracy: 0.4220 - factorized_top_k/top_50_categorical_accuracy: 0.4875 - factorized_top_k/top_100_categorical_accuracy: 0.5182 - loss: 14203.7730 - regularization_loss: 0.0000e+00 - total_loss: 14203.7730
Epoch 3/3
2/2 [==============================] - 0s 108ms/step - factorized_top_k/top_1_categorical_accuracy: 0.1219 - factorized_top_k/top_5_categorical_accuracy: 0.3681